# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Изучаем общую информацию о данных

In [1]:
import pandas as pd

In [2]:
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Итак, имеется датасет, состоящий из >20k строк и 12 столбцов (атрибутов). 
1. Не во всех столбцах одинаковое количество значений, значит нужно будет обязательно найти и разобраться с пропусками. 
2. Названия столбцов выглядят хорошо, без пропусков и заглавных букв, здесь ничего менять не нужно.
3. Все значения в датасете числовые или строковые. 
4. В столбцах `education_id`, `family_status_id`, `debt` у нас идентификаторы. Они не очень интуитивны, поэтому расшифруем их на всякий случай, соотнеся с категориальными переменными: 
    - Образование: 0 - высшее, 1 - среднее, 4 - ученая степень
    - Семейное положение: 0 - женат/замужем, 1 - гражданский брак, 3 - в разводе
    - Имел ли задолженность: будем считать 0 - нет, 1 - да


## Предобработка данных

### Обработка пропусков

Вызвав метод `info()`, видим невооруженным взглядом, что пропуски есть в колонках `days_employed` и `total_income` (количество значений в них меньше общего количества строк).

Сначала посмотрим в каких атрибутах пропуски и сколько их. Далее попробуем определить, какой природы эти пропуски и есть ли в них какие-то паттерны, и дальше решим, как лучше их заполнить.


In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

2174 пропуска это немало, почти 10% всех строк - удалять их точно не стоит. Что еще интересно, абсолютно одинаковое число пропусков в столбцах `days_employed` и `total_income`. Проверим,..

In [6]:
df[df['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Похоже что всех строках, где пропущено значение трудового стажа, пропущено и значение дохода. Это может быть либо технической ошибкой выгрузки (например, если данные сводились в эту таблицу из разных источников и что-то не склеилось). А может быть и человеческим фактором, например, кредитный специалист не заполнил данные о трудовом стаже и доходах. В этом случае можно говорить о неслучайных пропусках.

В реальной жизни на этом этапе необходимо было бы задать вопросы коллегам, которые делали выгрузку. Но так как в учебном проекте этой опции не предусмотрено, будем работать с тем, что есть.

 - Прежде всего разберемся артефактом: почему-то многие значения в `days_employed` отрицательные. Нужно убедиться, что это не ошибка выгрузки. Посчитаем количество положительных и отрицательных значений по категориям трудящихся.

In [7]:
print('Положительные значения:')
print(df[df['days_employed'] > 0]['income_type'].value_counts())

Положительные значения:
пенсионер      3443
безработный       2
Name: income_type, dtype: int64


In [8]:
print('Отрицательные значения:')
df[df['days_employed'] < 0]['income_type'].value_counts()

Отрицательные значения:


сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
в декрете              1
предприниматель        1
Name: income_type, dtype: int64

Все `положительные значения` - в категории пенсионеров и безработных, т.е. тех кто на момент получения кредита не работал.

`Отрицательные значения` - во всех остальных категориях, т.е. на момент подачи заявки на кредит они уже работали в течение какого-то времени. Возможно, минус здесь означает, что отсчет стажа происходит от текущей даты (или от даты заявки на кредит) назад. В этом случае все логично, минусы - это не ошибка. Почему у пенсионеров и безработных значение положительное? Возможно, так предусмотрено, чтобы выделить их из остальных категорий.

 - Теперь заполним пропуски по столбцу медианными значениями, рассчитанными отдельно для каждой из категорий `income_type`.

In [9]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

#проверяю что все пропуски заполнены
print('Осталось пропусков в "days_employed":', df['days_employed'].isnull().sum())

#sanity check: посмотрим сколько в среднем лет стажа приходится на каждую категорию и средний возраст
experience = df.groupby('income_type')['days_employed','dob_years'].median()
experience['years_employed'] = experience['days_employed'] / 261 # стаж в годах, если в году 261 рабочий день
experience

Осталось пропусков в "days_employed": 0


,days_employed,dob_years,years_employed
income_type,,,
безработный,366413.652744,38.0,1403.883727
в декрете,-3296.759962,39.0,-12.631264
госслужащий,-2689.368353,40.0,-10.304093
компаньон,-1547.382223,39.0,-5.928668
пенсионер,365213.306266,60.0,1399.284698
предприниматель,-520.848083,42.5,-1.995587
сотрудник,-1574.202821,39.0,-6.031428
студент,-578.751554,22.0,-2.217439


Ок, вроде бы все в порядке, средний стаж работы относительно среднего возраста выглядит правдоподобно. 

 - Теперь таким же методом заполним пропуски медианными значениями в столбце `total_income`, но только теперь группировку сделаем одновременно по типу занятости и уровню образования (ведь доход может сильно отличаться у высококвалифицированных специалистов и у разнорабочих).

In [10]:
# но сначала приведем все значения в education  к нижнему регистру, 
# чтобы данные не сгруппировались отдельно для "ВЫСШЕЕ" и "высшее", например
df['education'] = df['education'].str.lower()

df['total_income'] = df['total_income'].fillna(df.groupby(['income_type','education'])['total_income'].transform('median'))

#проверим, все ли пропуски заполнились?
print('Осталось пропусков в столбце "total_income":', df['total_income'].isnull().sum())

Осталось пропусков в столбце "total_income": 0


- sanity check: посмотрим какие получились значения доходов в разрезе образования и типа занятости

In [11]:
income_pivot_check = df.pivot_table(index='income_type', columns='education', values='total_income', aggfunc='median')
income_pivot_check

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
income_type,,,,,
безработный,202722.511368,NaN,NaN,59956.991984,NaN
в декрете,NaN,NaN,NaN,53829.130729,NaN
госслужащий,172511.107016,148339.290825,160592.345303,136652.970357,111392.231107
компаньон,201785.400018,136798.905143,179867.152890,159070.690289,NaN
пенсионер,144240.768611,102598.653164,120136.896353,114842.854099,177088.845999
предприниматель,499163.144947,NaN,NaN,NaN,NaN
сотрудник,165640.744634,125994.910603,151308.937846,136555.108821,198570.757322
студент,98201.625314,NaN,NaN,NaN,NaN


Сотрудник с высшим образованием имеет медианный доход выше, чем сотрудник с начальным образованием. Такая же ситуация с госслужащими и пенсионерами. Значит, решение посчитать медианный доход не только по типу занятости, но и по уровню образования, было верным.

**Вывод**

Итак, мы заполнили пропуски в количественных переменных медианными значениями, но не просто по всему столбцу, а предварительно сгруппировав данные:
- для стажа работы заполнили медианными значениями, рассчитанными в разрезе типа занятости, 
- а для ежемесячного дохода по типу занятости и уровню образования. 
Так мы уменьшили величину отклонения данных от (неизвестных нам) реальных значений.

### Замена типа данных

Еще раз взглянем на типы данных в датафрейме.

In [12]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

 - **Переведем стаж работы в целые числа,** 
ведь вряд ли кому-то понадобится точное значение до тысячной доли дня.

In [13]:
df['days_employed'] = df['days_employed'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 - **Переведем значения по столбцу gender в числа,**
пусть 2 = женщина (F), 1 = мужчина (M)

In [14]:
df['gender'] = df['gender'].map({'F': 2, 'M': 1})

Теперь приведем значения к целочисленным. Во избежание ошибок воспользуемся конструкцией `try-except`.

In [15]:
errors = []
for row in df['gender']:
    try:
        row = int(row)
    except:
        errors.append(row)

print(errors)

[nan]


Отлично! Ошибок не обнаружено. Проверим, сколько у нас наблюдений по каждому полу

In [16]:
df['gender'].value_counts()

2.0    14236
1.0     7288
Name: gender, dtype: int64

**Вывод**

Всего мы изменили тип данных в 2 столбцах: 

  - в days_employed заменили значения с десятичными дробями на целые числа, так таблица выглядит аккуратнее и логичнее считать стаж в полных днях работы. 
  - в gender заменили вещественные значения на целочисленные. Для того чтобы отловить неожиданные ошибки, применили метод try-except, и  убедились, что ошибок в значениях столбца не было, что не может не радовать.

Во всех остальных столбцах тип данных соответствует нашим целям анализа, поэтому больше ничего менять не будем.

### Обработка дубликатов

Найдем общее количество дубликатов в датафрейме и удалим их.

In [17]:
df.duplicated().sum()

71

In [18]:
dupes = df[df.duplicated() == True].sort_values(by=['purpose','dob_years'])
dupes

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13639,0,365213,64,среднее,1,женат / замужем,0,2.0,пенсионер,0,114842.854099,автомобиль
16378,0,-1574,46,среднее,1,женат / замужем,0,2.0,сотрудник,0,136555.108821,высшее образование
8583,0,365213,58,высшее,0,Не женат / не замужем,4,2.0,пенсионер,0,144240.768611,дополнительное образование
20702,0,365213,64,среднее,1,женат / замужем,0,2.0,пенсионер,0,114842.854099,дополнительное образование
18428,0,365213,64,среднее,1,женат / замужем,0,2.0,пенсионер,0,114842.854099,жилье
...,...,...,...,...,...,...,...,...,...,...,...,...
18521,0,-1574,56,среднее,1,гражданский брак,1,2.0,сотрудник,0,136555.108821,сыграть свадьбу
3290,0,365213,58,среднее,1,гражданский брак,1,2.0,пенсионер,0,114842.854099,сыграть свадьбу
5557,0,365213,58,среднее,1,гражданский брак,1,2.0,пенсионер,0,114842.854099,сыграть свадьбу
17338,0,365213,64,среднее,1,гражданский брак,1,2.0,пенсионер,0,114842.854099,сыграть свадьбу


Окей, у нас есть клиенты, данные о которых внесены повторно или даже несколько раз. Если исключить вариант, что это может быть случайностью, так как данные анонимизированные и у нас нет никакого  ID для клиентов, по которому мы могли бы отфильтровать уникальные значения, остается только догадываться, была ли это техническая ошибка и в чем ее причина. Но будем считать, что это действительно дубликаты.

Перед тем как применить метод drop_duplicates, можно еще привести все строки к нижнему регистру, и посмотреть, сколько тогда будет дубликатов. Так как мы уже ранее делали группировку по `income_type` и `education`, и приводили `education` к нижнем регистру, то знаем, что в них дубликатов уже нет. Остается `family_status` и `purpose`.

In [19]:
df['family_status'] = df['family_status'].str.lower()
df['purpose'] = df['purpose'].str.lower()

df.duplicated().sum()

71

Дубликатов по-прежнему 71. Смело удаляем их.

In [20]:
df = df.drop_duplicates().reset_index(drop=True)
df.shape

(21454, 12)

**Вывод**

После приведения всех строковых значений к нижнему регистру число дубликатов не увеличилось. Хотя мы помним, что в самом начале мы также приводили к нижнему регистру столбец `education` и там было очень много разных вариаций регистров. Можно сделать вывод, что часть дубликатов появилась из-за человеческой невнимательности - данные об уровне образования вводились в разных регистрах, а часть дубликатов можно списать на ошибки выгрузки.

### Лемматизация: анализируем наиболее частые цели кредита

Попробуем узнать, на что чаще всего берут кредит у нашего заказчика. Для этого сначала выделим леммы в значениях цели кредита.

In [21]:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(' '.join(df['purpose']))
lemmas

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'недвижимость',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',

Посчитаем, какие самые популярные леммы в нашем датасете.

In [22]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


**Вывод**

Как видно из подсчета частотности по леммам, чаще всего люди берут кредит на `недвижимость, жилье, автомобиль или образование`.

Такие леммы как `покупка` и `строительство` обозначают действия и встречаются в связке с существительными, поэтому их не считаем за отдельную категорию.

### Категоризация данных


#### Присвоение уровня риска в зависимости от дохода и занятости

Разделим все наблюдения на группы по уровню дохода и типу занятости и присвоим им категории "риска", т.е. клиенты с низким доходом и безработные будут наименее платежеспособными и самыми рискованными для банка, а клиенты с высоким доходом наименее рискованными. 

**Шаг 1.** Создадим новый датафрейм, в котором будут только необходимые нам для анализа данные: доход, тип занятости и наличие долга.

In [23]:
credit_risk = df[['total_income', 'income_type', 'debt']]
#добавляю столбец id, чтобы можно было построить сводную таблицу
credit_risk = credit_risk.reset_index()
names = credit_risk.columns.tolist()
names[0] = 'id'
credit_risk.columns = names
credit_risk.head()

,id,total_income,income_type,debt
0,0,253875.639453,сотрудник,0
1,1,112080.014102,сотрудник,0
2,2,145885.952297,сотрудник,0
3,3,267628.550329,сотрудник,0
4,4,158616.077870,пенсионер,0


**Шаг 2.** Разделим все значения столбца доходов на равные группы. Пусть будет 5 групп с идентификаторами от 1 до 5, где 1 - группа с доходом в нижнем 20% квантиле, а 5 - верхний квантиль с доходом выше, чем у остальных 80% клиентов.

In [24]:
bin_labels = range(1,6)
credit_risk['income_group'] = pd.qcut(credit_risk['total_income'], q=[0, .2, .4, .6, .8, 1], labels=bin_labels)
credit_risk.head()

,id,total_income,income_type,debt,income_group
0,0,253875.639453,сотрудник,0,5
1,1,112080.014102,сотрудник,0,2
2,2,145885.952297,сотрудник,0,3
3,3,267628.550329,сотрудник,0,5
4,4,158616.077870,пенсионер,0,3


In [25]:
credit_risk.groupby('income_group').mean()

,id,total_income,debt
income_group,,,
1,10736.713587,75426.250441,0.080168
2,10739.833139,115218.720317,0.083663
3,10768.964336,145295.040944,0.089977
4,10596.150548,184183.233758,0.082032
5,10790.848287,307139.664023,0.069914


Теперь, сгруппировав наши данные по уровню дохода от 1 до 5, можно увидеть а) средний доход по каждому квантилю и б) среднюю долю клиентов, имевших задолженности по кредиту.

Посмотрим на уровни доходов клиентов в разрезе типа занятости.

In [26]:
income_pivot = credit_risk.pivot_table(index='income_type', columns='income_group', values='id', aggfunc='count')
income_pivot

income_group,1,2,3,4,5
income_type,,,,,
безработный,1.0,NaN,NaN,1.0,NaN
в декрете,1.0,NaN,NaN,NaN,NaN
госслужащий,278.0,260.0,276.0,332.0,311.0
компаньон,556.0,761.0,986.0,1237.0,1538.0
пенсионер,1246.0,1077.0,527.0,529.0,450.0
предприниматель,NaN,NaN,NaN,NaN,2.0
сотрудник,2208.0,2193.0,2501.0,2192.0,1990.0
студент,1.0,NaN,NaN,NaN,NaN


Большинство пенсионеров относятся к низким группам доходов 1 и 2. Большинство компаньонов относятся к группам 4 и 5. Среди сотрудников распределение по уровню доходов равномерное, и больше всего людей в средней группе (3). По студентам и безработным у нас недостаточно наблюдений, чтобы проследить закономерности.

**Шаг 3.** Добавим в таблицу credit_risk категории риска.

К группе **Высокого риска** будут относиться: 
   - безработные, в декрете, студенты и пенсионеры с уровнем дохода 1-3
   - все прочие группы с уровнем дохода в нижнем 20% квантиле
   
**Средний риск**: сотрудники, предприниматели, госслужащие, компаньоны с уровнем дохода 2-3

**Низкий риск**: все клиенты с уровнем дохода 4-5

In [27]:
def risk_level(row):
    income = row['income_group']
    work = row['income_type']
    not_working = ['пенсионер', 'в декрете', 'безработный', 'студент']
    working = ['госслужащий', 'компаньон', 'предприниматель', 'сотрудник']

    if income >= 4:
        return 'низкий риск'
    else:
        if work in not_working:
            return 'высокий риск'
        elif work in working:
            if income == 1:
                return 'высокий риск'
            else:
                return 'средний риск'
    
    
credit_risk['risk_level'] = credit_risk.apply(risk_level, axis=1)
credit_risk.head(10)

,id,total_income,income_type,debt,income_group,risk_level
0,0,253875.639453,сотрудник,0,5,низкий риск
1,1,112080.014102,сотрудник,0,2,средний риск
2,2,145885.952297,сотрудник,0,3,средний риск
3,3,267628.550329,сотрудник,0,5,низкий риск
4,4,158616.077870,пенсионер,0,3,высокий риск
5,5,255763.565419,компаньон,0,5,низкий риск
6,6,240525.971920,компаньон,0,5,низкий риск
7,7,135823.934197,сотрудник,0,3,средний риск
8,8,95856.832424,сотрудник,0,1,высокий риск
9,9,144425.938277,сотрудник,0,3,средний риск


Функция работает корректно. Подсчитаем значения по каждой категории риска.

In [28]:
credit_risk['risk_level'].value_counts()

низкий риск     8582
средний риск    6977
высокий риск    5895
Name: risk_level, dtype: int64

**Вывод**

- Больше всего наблюдений оказалось в категории низкого риска и меньше всего - высокого. Так и должно быть, банк более заинтересован в тех клиентах, которые с большей вероятностью вернут кредит в срок.
- В то же время разброс величин по категориям небольшой. Если кредитный отдел захочет применить более строгие критерии оценки рисков по доходу, больше клиентов могут попасть в категории высокого или низкого риска. Все эти критерии, конечно, необходимо будет обсудить с заказчиком.

#### Категоризация по целям кредита

**Шаг 1:** Чтобы легче было сгруппировать данные по целям кредита, сначала выделим леммы в столбце `purpose`, и запишем их в новый столбец `lemm`

In [29]:
df['lemm'] = df.purpose.map(m.lemmatize)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm
0,1,-8437,42,высшее,0,женат / замужем,0,2.0,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,2.0,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,1.0,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,1.0,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,2.0,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Шаг 2:** Отлично, мы добавили леммы в новый столбец `lemm`. Теперь из него возьмем только те ключевые слова, которые максимально хорошо описывают, на что был выдан кредит: жилье, автомобиль, и т.д. Это и будут наши категории по целям кредита.

Для каждой строки будем искать ключевые слова в столбце с леммами и присваивать соответствующую категорию в столбце `purpose_category`.

In [30]:
def category(row):
    if 'автомобиль' in row['lemm']:
        return 'автомобиль'
    if 'свадьба' in row['lemm']:
        return 'свадьба'
    if 'жилье' in row['lemm']:
        return 'жилье'
    if 'недвижимость' in row['lemm']:
        return 'недвижимость'
    if 'образование' in row['lemm']:
        return 'образование'
    if 'ремонт' in row['lemm']:
        return 'ремонт'
    else:
        return 'другое'

df['purpose_category'] = df.apply(category, axis = 1)

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,2.0,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,-4024,36,среднее,1,женат / замужем,0,2.0,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,1.0,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,-4124,32,среднее,1,женат / замужем,0,1.0,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,2.0,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,-926,27,высшее,0,гражданский брак,1,1.0,компаньон,0,255763.565419,покупка жилья,"[покупка, , жилье, \n]",жилье
6,0,-2879,43,высшее,0,женат / замужем,0,2.0,компаньон,0,240525.971920,операции с жильем,"[операция, , с, , жилье, \n]",жилье
7,0,-152,50,среднее,1,женат / замужем,0,1.0,сотрудник,0,135823.934197,образование,"[образование, \n]",образование
8,2,-6929,35,высшее,0,гражданский брак,1,2.0,сотрудник,0,95856.832424,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,1.0,сотрудник,0,144425.938277,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье


**Шаг 3:** Посмотрим, какие категории у нас получились, и какая доля приходится на каждую из категорий.

In [31]:
df['purpose_category'].value_counts() / len(df)

недвижимость    0.296029
жилье           0.207887
автомобиль      0.200708
образование     0.187051
свадьба         0.108325
Name: purpose_category, dtype: float64

**Вывод**

Итак, мы с помощью лемматизации выделили более общие категории целей кредита и каждой строке присвоили соответствующую категорию.

## Группировка данных и поиск взаимосвязей


### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Чтобы ответить на этот вопрос, построим сводную таблицу методом `pivot_table()`, включив в нее данные по столбцам `children` и `debt`.
- сначала добавим в `df` столбец `id`, чтобы можно было по нему посчитать частотность:

In [32]:
df = df.reset_index()
col_names = df.columns.tolist()
col_names[0] = 'id'
df.columns = col_names
df.head()

,id,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm,purpose_category
0,0,1,-8437,42,высшее,0,женат / замужем,0,2.0,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,1,-4024,36,среднее,1,женат / замужем,0,2.0,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,2,0,-5623,33,среднее,1,женат / замужем,0,1.0,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,3,-4124,32,среднее,1,женат / замужем,0,1.0,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,4,0,340266,53,среднее,1,гражданский брак,1,2.0,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


- посмотрим распределение по количеству детей

In [33]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

*Сюрприз!* Почему-то в 47 строках количество детей равно -1. Так как такое невозможно, заменим эти значения на 0.

In [34]:
df.loc[df['children'] == -1, 'children'] = 0
df['children'].value_counts()

0     14138
1      4808
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

А вот и еще один артефакт: у 76 клиентов записано аж по 20 детей. Эти данные тоже скорее всего не соответствуют действительности, поэтому будем рассматривать их не как выброс, а как ошибку при вводе данных. Возможно, сотрудник в программе хотел заполнить "2", но не убрал ноль, и получилось 20, или это был баг в программе, где вносились данные.

Заменим все значения "20" на "2".

In [35]:
df.loc[df['children'] == 20, 'children'] = 2

- От аномальных значений избавились, теперь можно строить сводную таблицу

In [36]:
children = df.pivot_table(index='children', columns='debt', values='id', aggfunc='count')
children

debt,0,1
children,,
0,13074.0,1064.0
1,4364.0,444.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [37]:
children2 = df.pivot_table(index='children', values='debt', aggfunc='mean')
children2

,debt
children,
0,0.075258
1,0.092346
2,0.094925
3,0.081818
4,0.097561
5,0.000000


- добавим в таблицу долю должников в процентах по каждой строке

In [38]:
children['total'] = children[0] + children[1]
children['no_debt_percentage'] = (children[0] / children['total'])
children['debt_percentage'] = (children[1] / children['total'])
children.style.format({'debt_percentage': '{:.2%}', 'no_debt_percentage': '{:.2%}'})

debt,0,1,total,no_debt_percentage,debt_percentage
children,,,,,
0,13074,1064,14138,92.47%,7.53%
1,4364,444,4808,90.77%,9.23%
2,1926,202,2128,90.51%,9.49%
3,303,27,330,91.82%,8.18%
4,37,4,41,90.24%,9.76%
5,9,nan,nan,nan%,nan%


**Вывод**

- Можно заключить, что те, у кого нет детей, немного чаще возвращают кредит в срок. Но эта зависимость не сильная: если посмотреть на данные в строке где 3 детей, процент возврата кредита в срок здесь будет отличаться меньше, чем на 1%
- Сильной зависимости между количеством детей и возвратом кредита в срок также не обнаружено: те, у кого всего 1 ребенок и те, у кого их 4, возвращают кредит в срок в 90% случаев.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Снова построим сводную таблицу методом `pivot_table()`, на этот раз включив в нее данные по столбцам `family_status` и `debt`.

In [39]:
family = df.pivot_table(index='family_status', columns='debt', values='id', aggfunc='count')
family

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


In [40]:
family2 = df.pivot_table(index='family_status', values='debt', aggfunc='mean')
family2.sort_values(by='debt', ascending=False).style.format({'debt':'{:.2%}'})

,debt
family_status,
не женат / не замужем,9.75%
гражданский брак,9.35%
женат / замужем,7.55%
в разводе,7.11%
вдовец / вдова,6.57%


In [41]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Посчитаем доли по каждой строке

In [42]:
family['debt_percentage'] = family[1] / (family[0] + family[1])
family['no_debt_percentage'] = family[0] / (family[0] + family[1])
family = family.sort_values(by='debt_percentage', ascending=False)
family.style.format({'debt_percentage': '{:.2%}', 'no_debt_percentage': '{:.2%}'})

debt,0,1,debt_percentage,no_debt_percentage
family_status,,,,
не женат / не замужем,2536,274,9.75%,90.25%
гражданский брак,3763,388,9.35%,90.65%
женат / замужем,11408,931,7.55%,92.45%
в разводе,1110,85,7.11%,92.89%
вдовец / вдова,896,63,6.57%,93.43%


**Вывод**

- Можно заметить зависимость между семейным положением и возвратом кредита в срок: холостые и те, кто состоит в гражданском браке, в среднем на 2% чаще имеют просрочку по кредиту, чем женатые и замужние или те, кто в разводе.
- Чаще всего возвращают кредит в срок вдовцы.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу, включив в нее данные по столбцам `debt` и `income_group` из таблицы `credit_risk`, которую мы создали ранее.

In [43]:
income_pivot = credit_risk.pivot_table(index='income_group', columns='debt', values='id', aggfunc='count')
income_pivot

debt,0,1
income_group,,
1,3947,344
2,3932,359
3,3904,386
4,3939,352
5,3991,300


Добавим процентные доли для каждого уровня дохода.

In [44]:
income_pivot['debt_percentage'] = income_pivot[1] / (income_pivot[0] + income_pivot[1])
income_pivot['no_debt_percentage'] = income_pivot[0] / (income_pivot[0] + income_pivot[1])
income_pivot = income_pivot.sort_values(by='debt_percentage', ascending=False)
income_pivot.style.format({'debt_percentage': '{:.2%}', 'no_debt_percentage': '{:.2%}'})

debt,0,1,debt_percentage,no_debt_percentage
income_group,,,,
3,3904,386,9.00%,91.00%
2,3932,359,8.37%,91.63%
4,3939,352,8.20%,91.80%
1,3947,344,8.02%,91.98%
5,3991,300,6.99%,93.01%


In [45]:
income_pivot2 = credit_risk.pivot_table(index='income_group', values='debt', aggfunc='mean')
income_pivot2.sort_values(by='debt', ascending=False)
income_pivot2.style.format({'debt' : '{:.2%}'})

,debt
income_group,
1,8.02%
2,8.37%
3,9.00%
4,8.20%
5,6.99%


**Вывод**

- Невероятно, но факт: клиенты, относящиеся к 80му квантилю по размеру дохода, то есть имеющие самый высокий доход, возвращают кредит в срок лишь на 1% чаще, чем самые бедные клиенты, относящиеся к нижнему 20 квантилю.
- Среди остальных категорий сильной зависимости между уровнем дохода и возвратом в срок не наблюдается.

### Как разные цели кредита влияют на его возврат в срок?

Чтобы легче было сгруппировать данные по целям кредита, сначала выделим леммы в столбце `purpose`

Сделаем сводную таблицу по целям и возврату в срок.

In [46]:
purpose_pivot = df.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
purpose_pivot.sort_values(by='debt', ascending=False)
purpose_pivot.style.format({'debt' : '{:.2%}'})

,debt
purpose_category,
автомобиль,9.36%
жилье,6.91%
недвижимость,7.46%
образование,9.22%
свадьба,8.00%


**Вывод**

Как видно из сводной таблицы, чаще всего не возвращают в срок кредиты на автомобиль и образование: более чем в 9% наблюдений.
Чуть лучше ситуация по кредитам на свадьбу (~7.9% не вернули в срок) и недвижимость (7.4%). А меньше всего просрочек случается по кредитам на жилье.

## Общий вывод

- **В первой части проекта** мы рассмотрели исходные данные, оценили объем датасета и основные типы данных.


- **Во второй части** были проведены различные операции предобработки данных:
    1. Поиск и заполнение пропусков в переменных `days_employed` и `total_income`. Для уменьшения искажения данных пропуски заполнялись медианными значениями по подгруппам в разрезе типа занятости и уровня образования.
    2. Замена типа данных: в столбце `days_employed` дробные числа были заменены на целые, и в `gender` вместо строк мы присвоили целочисленные значения. Хотя в данном проекте эти 2 переменные никак не использовались, в будущем это облегчит работу, если например придет дополнительный запрос от заказчика.
    3. Поиск дубликатов с учетом регистра и их удаление.
    4. Лемматизация значений по целям кредита и подсчет частотности лемм, в результате чего мы выяснили, что самыми популярными целями кредита являются недвижимость, жилье, автомобиль и образование.
    5. Категоризация данных: здесь мы разделили заемщиков на три категории по уровню платежеспособности, или, с точки зрения банка, по уровню риска и посмотрели, увидели, что больше всего заемщиков у нас в категории низкого риска, то есть наша скоринговая система на верном пути.
    
    
- **В третьей части проекта** данные были организованы в сводные таблицы, чтобы попытаться выяснить, как различные факторы влияют на возврат кредита в срок.
    1. Наличие детей слабо влияет на возврат кредита в срок. Бездетные клиенты возвращают кредит в срок примерно на 2% чаще, чем те, у кого есть дети. При этом те, у кого трое детей, чаще возвращают кредит, чем те, у кого всего один ребенок.
    2. Семейное положение: с наибольшей вероятностью вернут кредит в срок вдовцы и те, кто в разводе. Реже возвращают в срок холостые и незамужние клиенты.
    3. Уровень дохода: клиенты с самым высоким уровнем дохода возвращают кредит в срок всего на 1% чаще, чем клиенты с самым низким уровнем дохода. Можно предположить, что на возврат кредита влияет главным образом не финансовое положение клиента, а какие-то другие факторы. К тому же у нас нет данных о размерах кредита, возможно менее обеспеченные люди берут в кредит меньшие суммы и отдают их быстрее.
    4. Разные цели кредита: чаще всего просрочки случаются по кредитам на автомобили и образование, а реже всего - по жилищным кредитам.